In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [13]:
teste = pd.read_csv("datasets/Classification/Corona_NLP_test.csv")
treino = pd.read_csv("datasets/Classification/Corona_NLP_train.csv",encoding='latin-1')

data = pd.concat([teste,treino])
data = data.sample(frac=0.1, random_state=42)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4496 entries, 29256 to 6491
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   UserName       4496 non-null   int64 
 1   ScreenName     4496 non-null   int64 
 2   Location       3571 non-null   object
 3   TweetAt        4496 non-null   object
 4   OriginalTweet  4496 non-null   object
 5   Sentiment      4496 non-null   object
dtypes: int64(2), object(4)
memory usage: 245.9+ KB


In [14]:
data = data.drop(columns=[ "UserName","ScreenName","Location","TweetAt"], axis=1)

In [15]:
def categorize_sentiment(score):
    if score == 'Negative':
        return "Negative"
    elif score == 'Extremely Negative':
        return "Negative"
    elif score == 'Positive':
        return "Positive"
    elif score == 'Extremely Positive':
        return "Positive"
    else:
        return "Neutral"
    
    
data['Sentiment'] = data['Sentiment'].apply(categorize_sentiment)

In [16]:
# Fazer TOKENIZACAO das palavras

import nltk

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [17]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def get_all_str(tweet):
    sentence = ''
    for words in tweet:
        sentence += words
    sentence = sentence.lower()
    return sentence
def get_str(lst):
    sentence = ''
    for char in lst:
        sentence += char+' '
    sentence = sentence.lower()
    return sentence

def get_word(text): 
    result = nltk.RegexpTokenizer(r'\w+').tokenize(text.lower())
    return result

def get_hashtag(text):
    result = nltk.RegexpTokenizer(r'(?<=#)\w+').tokenize(text.lower())
    return result

def get_mention(text):
    result = nltk.RegexpTokenizer(r'(?<=@)\w+').tokenize(text.lower())
    return result 

def stopword_list(stop):
    lst = stopwords.words('english')
    for stopword in stop:
        lst.append(stopword)
    return lst

def remove_stopword(stopwords, lst):    
    stoplist = stopword_list(stopwords)
    txt = ''
    for idx in range(len(lst)):
        txt += lst[idx]
        txt += '\n'
    cleanwordlist = [word for word in txt.split() if word not in stoplist] 
#     print(stoplist)
    return cleanwordlist

def pos_remove_noun(tagged):
    remove_noun = [word for word,pos in tagged if pos not in ['NN','NNS','NNP','NNPS']]
    txt = ''
    for i in range(len(remove_noun)):
        txt += remove_noun[i]
        txt += '\n'
    return cleanwordlist

def lemmatization(words):
    lemm = WordNetLemmatizer()
    tokens = [lemm.lemmatize(word) for word in words]
#     txt= ' '.join(tokens)
    return tokens

In [18]:
def preprocessing(text):
    words = get_word(text)
    lemma = lemmatization(words)
    removed = remove_stopword(['http','ly'],lemma)
    return removed

data['OriginalTweet'] = data['OriginalTweet'].apply(preprocessing)
data['OriginalTweet'] = data['OriginalTweet'].apply(get_str)

In [19]:
X = data.drop(columns=["Sentiment"])
y = data["Sentiment"]

In [20]:
X

,OriginalTweet
29256,worried empty grocery store shelf limited food...
29766,co 45fpphe6e3 victoria help food production 1s...
28909,breaking news hero nhsuk worker ha â attackedâ...
9454,role play pandemic today helping drive testing...
34039,currently spaced queue iâ ever buy essential i...
...,...
35044,covid 19 pandemic store usually offer free shi...
1812,latimes grocery store letting 100 people time ...
1157,wa toilet paper went grocery store cdc recomme...
3037,keep coronavirus bay mygovindia 1 wash hand so...


In [21]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer


Tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1,2), norm='l2' )
X = Tfidf.fit_transform(data["OriginalTweet"]).toarray()




In [22]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [23]:
# Load the Iris dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
# Scale the data

scaler = StandardScaler(with_mean=False)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [25]:
from sklearn.neural_network import MLPClassifier

# Initialize the MLPClassifier
mlp_classifier = MLPClassifier(hidden_layer_sizes=(50, 50), max_iter=1000, random_state=42)


In [26]:

# Fit the model to the training data
mlp_classifier.fit(X_train, y_train)


MLPClassifier(hidden_layer_sizes=(50, 50), max_iter=1000, random_state=42)

In [27]:

# Make predictions
y_pred = mlp_classifier.predict(X_test)

In [28]:
from sklearn.metrics import accuracy_score

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)*100

# Print the accuracy
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 64.11
